# Task 3 - Point Cloud Scenes

Visualizing large-scale point cloud scenes is important when qualitatively evaluating predictions from a machine learning model or exploring what is happening in an outdoor scene. To this end, using Python and Open3D (http://www.open3d.org/docs/release/getting_started.html), implement a visualization of a LiDAR point cloud sequence from the SemanticKITTI dataset. Your implementation should 1) allow you to navigate forward and backward in time using the left and right arrow keys, and 2) visualize the semantic labels for each point. An example visualization is shown in the following video (semantic labels on the left): https://www.youtube.com/watch?v=VRZGAK3XpX0.

In [1]:
import os
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Helper Functions

In [3]:
color_map = {
   0 : [0, 0, 0],
  1 : [0, 0, 255],
  10: [245, 150, 100],
  11: [245, 230, 100],
  13: [250, 80, 100],
  15: [150, 60, 30],
  16: [255, 0, 0],
  18: [180, 30, 80],
  20: [255, 0, 0],
  30: [30, 30, 255],
  31: [200, 40, 255],
  32: [90, 30, 150],
  40: [255, 0, 255],
  44: [255, 150, 255],
  48: [75, 0, 75],
  49: [75, 0, 175],
  50: [0, 200, 255],
  51: [50, 120, 255],
  52: [0, 150, 255],
  60: [170, 255, 150],
  70: [0, 175, 0],
  71: [0, 60, 135],
  72: [80, 240, 150],
  80: [150, 240, 255],
  81: [0, 0, 255],
  99: [255, 255, 50],
  252: [245, 150, 100],
  256: [255, 0, 0],
  253: [200, 40, 255],
  254: [30, 30, 255],
  255: [90, 30, 150],
  257: [250, 80, 100],
  258: [180, 30, 80],
  259: [255, 0, 0]
}

def get_path(task_number, *args):
    notebook_path = os.path.abspath(f"Task_{task_number}.ipynb")
    return os.path.join(os.path.dirname(notebook_path), *args)


def label_to_color(label: np.uint32):
    global color_map
    
    if label not in color_map:
        return np.array([0, 0, 0])
    
    return np.array(color_map[label])


def load_point_clouds(save_dir):
    pcds = []
    
    for filename in sorted(os.listdir(save_dir)):
        pcd = o3d.io.read_point_cloud(os.path.join(save_dir, filename))
        pcds.append(pcd)
    
    return pcds


def bin_to_point_cloud(bin_file_path, label_file_path, show_instance_labels=False):
    # Load the .bin data into a numpy array
    point_cloud_np = np.fromfile(bin_file_path, dtype=np.float32)
    labels = np.fromfile(label_file_path, dtype=np.uint32)
    
    # Reshape the numpy array to Nx4 (x, y, z, intensity)
    point_cloud_np = point_cloud_np.reshape(-1, 4)
    labels = labels.reshape(-1, 1)
    
    # Separate the semantic labels and the instance labels
    lables =  labels >> 16  if show_instance_labels else labels & 0xFFFF
    
    # Create an Open3D point cloud object
    point_cloud_o3d = o3d.geometry.PointCloud()
    
    # Set the points of the point cloud from the numpy array
    point_cloud_o3d.points = o3d.utility.Vector3dVector(point_cloud_np[:, :3])
    point_cloud_o3d.colors = o3d.utility.Vector3dVector(np.array([label_to_color(int(label[0])) for label in lables]))
    
    return point_cloud_o3d

# Generate o3d Point Clouds From Dataset


### 1. Read .bin file
### 2. Convert into o3d Point Cloud
### 3. Save to .ply file

In [15]:
SAVE_DIR = get_path(3, "save", "normal")
SEQ_DIR = get_path(3, "data", "normal", "dataset", "sequences")
SEQ = ["00"]

for seq in SEQ:
  velodyne_dir = os.path.join(SEQ_DIR, seq, "velodyne")
  labels_dir = os.path.join(SEQ_DIR, seq, "labels")
  
  for file_name in sorted(os.listdir(velodyne_dir)):
    if file_name.endswith(".bin"):
      bin_file_path = os.path.join(velodyne_dir, file_name)
      label_file_path = os.path.join(labels_dir, file_name[:-4] + ".label")
      
      save_file_name = f"{seq}_{file_name[:-4]}.ply"
      save_file_path = os.path.join(SAVE_DIR, seq, save_file_name)
      
      # check if file already exists
      if os.path.isfile(save_file_path):
        print(f"Point Cloud for {save_file_name} already exists. Skipping...")
        continue
      
      if not os.path.exists(os.path.join(SAVE_DIR, seq)):
        os.makedirs(os.path.join(SAVE_DIR, seq))
      
      # Convert to Open3D point cloud
      print("Generating Voxel Grid for " + save_file_name)
      pcd = bin_to_point_cloud(bin_file_path, label_file_path)
      
      # Save point cloud to file
      print(f"Saving Point Cloud to {save_file_path}")
      o3d.io.write_point_cloud(save_file_path, pcd)



Point Cloud for 00_000000.ply already exists. Skipping...
Point Cloud for 00_000001.ply already exists. Skipping...
Point Cloud for 00_000002.ply already exists. Skipping...
Point Cloud for 00_000003.ply already exists. Skipping...
Point Cloud for 00_000004.ply already exists. Skipping...
Point Cloud for 00_000005.ply already exists. Skipping...
Point Cloud for 00_000006.ply already exists. Skipping...
Point Cloud for 00_000007.ply already exists. Skipping...
Point Cloud for 00_000008.ply already exists. Skipping...
Point Cloud for 00_000009.ply already exists. Skipping...
Point Cloud for 00_000010.ply already exists. Skipping...
Point Cloud for 00_000011.ply already exists. Skipping...
Point Cloud for 00_000012.ply already exists. Skipping...
Point Cloud for 00_000013.ply already exists. Skipping...
Point Cloud for 00_000014.ply already exists. Skipping...
Point Cloud for 00_000015.ply already exists. Skipping...
Point Cloud for 00_000016.ply already exists. Skipping...
Point Cloud fo

KeyboardInterrupt: 

# Run Open3D Visualization

In [16]:
# Load all saved voxel grids
SAVE_DIR = get_path(3, "save", "normal", "00")
point_clouds = load_point_clouds(SAVE_DIR)

# Create a visualizer and window
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window(width=1000, height=1000)
view_control = vis.get_view_control()

# Add the first point cloud to the visualizer
vis.add_geometry(point_clouds[0])

# This index will keep track of the current point cloud
idx = 0


# Define the functions to be called when the left or right arrow key is pressed
def left_key_action(vis):
    global idx
    
    camera_parameters = vis.get_view_control().convert_to_pinhole_camera_parameters()

    idx = (idx - 1) % len(point_clouds)
    vis.clear_geometries()
    vis.add_geometry(point_clouds[idx])
    
    # Restore the camera parameters
    view_control.convert_from_pinhole_camera_parameters(camera_parameters, True)
    vis.update_renderer()
    
def right_key_action(vis):
    global idx

    camera_parameters = vis.get_view_control().convert_to_pinhole_camera_parameters()

    idx = (idx + 1) % len(point_clouds)
    vis.clear_geometries()
    vis.add_geometry(point_clouds[idx])
    
    # Restore the camera parameters
    view_control.convert_from_pinhole_camera_parameters(camera_parameters, True)
    vis.update_renderer()

# Bind the functions to the left and right arrow keys
vis.register_key_callback(262, right_key_action)  
vis.register_key_callback(263, left_key_action)  

# Start the visualization
vis.run()
vis.destroy_window()

[Open3D WARNING] Read PLY failed: unable to parse header.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
